In [1]:
import json
import pandas as pd
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense
from tensorflow.keras.optimizers import Adam

In [2]:
def isChinese(string):
    for ch in string:
        if u'\u4e00' <= ch <= u'\u9fff':
            return True
    
    return False

def init():
    lists = []
    with open('./translation2019zh/translation2019zh_train.json','r', encoding='utf-8') as dat_f:
        data = []
        for i,line in enumerate(dat_f):
            data = json.loads(line)

            if isChinese(data['chinese']) == True:
                if len(data['chinese'])<6:
                    lists.append(data)
                if (len(lists)+1)%100 == 0:
                    break
    
    df = pd.DataFrame(lists)
    df.to_csv('datafile.csv', encoding='utf-8', index=False)

init()

In [3]:
df = pd.read_csv('datafile.csv')
df['chinese'] = df['chinese'].apply(lambda x: '@' + x + '。')
en_data = df.english.values.tolist()
ch_data = df.chinese.values.tolist()

In [4]:
en_vocab = set(''.join(en_data)) # 生成字典
id2en = list(en_vocab)
en2id = {c:i for i,c in enumerate(id2en)}
ch_vocab = set(''.join(ch_data))
id2ch = list(ch_vocab)
ch2id = {c:i for i,c in enumerate(id2ch)}
print('\n英文:\n', en2id)
print('\n中文\n:', ch2id)


英文:
 {'t': 0, 'z': 1, ';': 2, 'J': 3, '=': 4, 'O': 5, 'y': 6, 'E': 7, ',': 8, '8': 9, 'r': 10, 'P': 11, 'o': 12, 'F': 13, 'T': 14, 'N': 15, 'K': 16, '"': 17, '）': 18, 'W': 19, 'w': 20, 'l': 21, '?': 22, 'I': 23, 'x': 24, 'e': 25, 'j': 26, 'D': 27, ' ': 28, 'f': 29, 'C': 30, 'X': 31, 'u': 32, 'V': 33, 'b': 34, '(': 35, 'S': 36, 'a': 37, 'c': 38, 'm': 39, 'M': 40, 'U': 41, ':': 42, 'R': 43, 'g': 44, '!': 45, '.': 46, ')': 47, 'h': 48, 'd': 49, 'G': 50, 'A': 51, 'i': 52, 'p': 53, 'q': 54, "'": 55, 'L': 56, 'B': 57, 's': 58, 'n': 59, '（': 60, 'Y': 61, 'v': 62, '…': 63, '-': 64, 'H': 65, 'k': 66}

中文
: {'氧': 0, '三': 1, '暴': 2, '魂': 3, '请': 4, '烤': 5, '聪': 6, '吗': 7, '换': 8, '驱': 9, '袭': 10, '母': 11, '。': 12, '洱': 13, '官': 14, '康': 15, '天': 16, '铜': 17, '于': 18, '?': 19, '托': 20, '采': 21, '楼': 22, '盐': 23, '魁': 24, '爱': 25, ' ': 26, '之': 27, '图': 28, '没': 29, '！': 30, '安': 31, '看': 32, '款': 33, '玻': 34, '直': 35, '标': 36, '深': 37, '如': 38, '怕': 39, '死': 40, '运': 41, '办': 42, '目': 43, '.': 44

In [5]:
def numData(en2id,ch2id,en_data):
    en_num_data = [[en2id[en] for en in line ] for line in en_data]
    ch_num_data = [[ch2id[ch] for ch in line] for line in ch_data]
    de_num_data = [[ch2id[ch] for ch in line][1:] for line in ch_data]
    print('char:', en_data[1])
    print('index:', en_num_data[1])
    max_encoder_seq_length = max([len(txt) for txt in en_num_data])
    max_decoder_seq_length = max([len(txt) for txt in ch_num_data])
    print('max encoder length:', max_encoder_seq_length)
    print('max decoder length:', max_decoder_seq_length)
    
    encoder_input_data = np.zeros((len(en_num_data), max_encoder_seq_length, len(en2id)), dtype='float32')
    decoder_input_data = np.zeros((len(ch_num_data), max_decoder_seq_length, len(ch2id)), dtype='float32')
    decoder_target_data = np.zeros((len(ch_num_data), max_decoder_seq_length, len(ch2id)), dtype='float32')
    
    for i in range(len(ch_num_data)):
        for t, j in enumerate(en_num_data[i]):
            encoder_input_data[i, t, j] = 1.
        for t, j in enumerate(ch_num_data[i]):
            decoder_input_data[i, t, j] = 1.
        for t, j in enumerate(de_num_data[i]):
            decoder_target_data[i, t, j] = 1.
            
    print('index data:\n', en_num_data[1])
    print('one hot data:\n', encoder_input_data[1])
    return encoder_input_data,decoder_input_data,decoder_target_data

nd = numData(en2id,ch2id,en_data)

char: She was possessed by a devil.
index: [36, 48, 25, 28, 20, 37, 58, 28, 53, 12, 58, 58, 25, 58, 58, 25, 49, 28, 34, 6, 28, 37, 28, 49, 25, 62, 52, 21, 46]
max encoder length: 75
max decoder length: 7
index data:
 [36, 48, 25, 28, 20, 37, 58, 28, 53, 12, 58, 58, 25, 58, 58, 25, 49, 28, 34, 6, 28, 37, 28, 49, 25, 62, 52, 21, 46]
one hot data:
 [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [6]:
EN_VOCAB_SIZE = len(en2id)
CH_VOCAB_SIZE = len(ch2id)
HIDDEN_SIZE = 256
LEARNING_RATE = 0.01
BATCH_SIZE = 20
EPOCHS = 250

In [7]:
def modeling():
    encoder_inputs = Input(shape=(None, EN_VOCAB_SIZE))
    encoder_h1, encoder_state_h1, encoder_state_c1 = LSTM(HIDDEN_SIZE, return_sequences=True, return_state=True)(encoder_inputs)
    encoder_h2, encoder_state_h2, encoder_state_c2 = LSTM(HIDDEN_SIZE, return_state=True)(encoder_h1)
    decoder_inputs = Input(shape=(None, CH_VOCAB_SIZE))
    lstm1 = LSTM(HIDDEN_SIZE, return_sequences=True, return_state=True)
    lstm2 = LSTM(HIDDEN_SIZE, return_sequences=True, return_state=True)
    decoder_dense = Dense(CH_VOCAB_SIZE, activation='softmax')
    
    decoder_h1, _, _ = lstm1(decoder_inputs, initial_state=[encoder_state_h1, encoder_state_c1])
    decoder_h2, _, _ = lstm2(decoder_h1, initial_state=[encoder_state_h2, encoder_state_c2])
    decoder_outputs = decoder_dense(decoder_h2)
    model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
    opt = Adam(lr=LEARNING_RATE, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    model.summary()
    model.fit([nd[0], nd[1]], nd[2],batch_size=BATCH_SIZE,epochs=EPOCHS,validation_split=0.)
    model.save('s2s.h5')
    encoder_model = Model(encoder_inputs, [encoder_state_h1, encoder_state_c1, encoder_state_h2, encoder_state_c2])
    decoder_state_input_h1 = Input(shape=(HIDDEN_SIZE,))
    decoder_state_input_c1 = Input(shape=(HIDDEN_SIZE,))
    decoder_state_input_h2 = Input(shape=(HIDDEN_SIZE,))
    decoder_state_input_c2 = Input(shape=(HIDDEN_SIZE,))
    decoder_h1, state_h1, state_c1 = lstm1(decoder_inputs, initial_state=[decoder_state_input_h1, decoder_state_input_c1])
    decoder_h2, state_h2, state_c2 = lstm2(decoder_h1, initial_state=[decoder_state_input_h2, decoder_state_input_c2])
    decoder_outputs = decoder_dense(decoder_h2)
    decoder_model = Model([decoder_inputs, decoder_state_input_h1, decoder_state_input_c1, decoder_state_input_h2, decoder_state_input_c2], [decoder_outputs, state_h1, state_c1, state_h2, state_c2])
    return encoder_model,decoder_model

model = modeling()

2021-08-20 10:36:53.806190: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")
2021-08-20 10:36:54.598399: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None, 67)]   0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, None, 281)]  0                                            
__________________________________________________________________________________________________
lstm (LSTM)                     [(None, None, 256),  331776      input_1[0][0]                    
__________________________________________________________________________________________________
lstm_2 (LSTM)                   [(None, None, 256),  550912      input_2[0][0]                    
                                                                 lstm[0][1]                   

5/5 [==============================] - 1s 200ms/step - loss: 0.6750 - accuracy: 0.6508
Epoch 61/250
5/5 [==============================] - 1s 206ms/step - loss: 0.6745 - accuracy: 0.6522
Epoch 62/250
5/5 [==============================] - 1s 199ms/step - loss: 0.6752 - accuracy: 0.6494
Epoch 63/250
5/5 [==============================] - 1s 204ms/step - loss: 0.6759 - accuracy: 0.6508
Epoch 64/250
5/5 [==============================] - 1s 197ms/step - loss: 0.6794 - accuracy: 0.6494
Epoch 65/250
5/5 [==============================] - 1s 210ms/step - loss: 0.6769 - accuracy: 0.6494
Epoch 66/250
5/5 [==============================] - 1s 198ms/step - loss: 0.6774 - accuracy: 0.6537
Epoch 67/250
5/5 [==============================] - 1s 198ms/step - loss: 0.6749 - accuracy: 0.6508
Epoch 68/250
5/5 [==============================] - 1s 199ms/step - loss: 0.6819 - accuracy: 0.6537
Epoch 69/250
5/5 [==============================] - 1s 199ms/step - loss: 0.6716 - accuracy: 0.6537
Epoch 70/250


5/5 [==============================] - 1s 203ms/step - loss: 1.3253 - accuracy: 0.4964
Epoch 142/250
5/5 [==============================] - 1s 200ms/step - loss: 1.2630 - accuracy: 0.4949
Epoch 143/250
5/5 [==============================] - 1s 200ms/step - loss: 1.1195 - accuracy: 0.5224
Epoch 144/250
5/5 [==============================] - 1s 202ms/step - loss: 1.0486 - accuracy: 0.5527
Epoch 145/250
5/5 [==============================] - 1s 200ms/step - loss: 1.0086 - accuracy: 0.5714
Epoch 146/250
5/5 [==============================] - 1s 210ms/step - loss: 0.9779 - accuracy: 0.5786
Epoch 147/250
5/5 [==============================] - 1s 219ms/step - loss: 0.9344 - accuracy: 0.5859
Epoch 148/250
5/5 [==============================] - 1s 235ms/step - loss: 0.9004 - accuracy: 0.5931
Epoch 149/250
5/5 [==============================] - 1s 222ms/step - loss: 0.8650 - accuracy: 0.6046
Epoch 150/250
5/5 [==============================] - 1s 222ms/step - loss: 0.8612 - accuracy: 0.6104
Epoc

5/5 [==============================] - 1s 212ms/step - loss: 0.4014 - accuracy: 0.6825
Epoch 223/250
5/5 [==============================] - 1s 210ms/step - loss: 0.4025 - accuracy: 0.6825
Epoch 224/250
5/5 [==============================] - 1s 199ms/step - loss: 0.3994 - accuracy: 0.6869
Epoch 225/250
5/5 [==============================] - 1s 213ms/step - loss: 0.4067 - accuracy: 0.6724
Epoch 226/250
5/5 [==============================] - 1s 208ms/step - loss: 0.3924 - accuracy: 0.6883
Epoch 227/250
5/5 [==============================] - 1s 210ms/step - loss: 0.3820 - accuracy: 0.6811
Epoch 228/250
5/5 [==============================] - 1s 232ms/step - loss: 0.3765 - accuracy: 0.6854
Epoch 229/250
5/5 [==============================] - 1s 241ms/step - loss: 0.3694 - accuracy: 0.6883
Epoch 230/250
5/5 [==============================] - 1s 271ms/step - loss: 0.3661 - accuracy: 0.6926
Epoch 231/250
5/5 [==============================] - 1s 206ms/step - loss: 0.3585 - accuracy: 0.6970
Epoc

In [8]:
def result():    
    for k in range(0,99):
        test_data = nd[0][k:k+1]
        h1, c1, h2, c2 = model[0].predict(test_data)
        target_seq = np.zeros((1, 1, CH_VOCAB_SIZE))
        target_seq[0, 0, ch2id['@']] = 1
        outputs = []
        while True:
            output_tokens, h1, c1, h2, c2 = model[1].predict([target_seq, h1, c1, h2, c2])
            sampled_token_index = np.argmax(output_tokens[0, -1, :])
            outputs.append(sampled_token_index)
            target_seq = np.zeros((1, 1, CH_VOCAB_SIZE))
            target_seq[0, 0, sampled_token_index] = 1
            if sampled_token_index == ch2id['。'] or len(outputs) > 10:
                break
        
        print(en_data[k])
        print(''.join([id2ch[i] for i in outputs]))

result()

Side-to-Side Movements.
撒母耳阿。
She was possessed by a devil.
撒母耳阿。
The majority was wrong last time.
没有应问。
Great talents flower late.
大器晚成。
Erhai Lake （in Yunnan Province）
洱海。
It strikes one as very strange.
我保证。
Did I divide right?
大东烤鸭。
I promise".
我是康康。
All was quiet and not a soul was to be seen.
车船齐发。
Folder wire glass;
撒母耳阿。
Steam bath;
撒母耳阿。
Package sealing.
请坐吧。
Mud on salt.
泥在盐上。
What street?
多么讽刺！。
You must be very quiet. Hold tight to me.
抱紧我。
Frozen ocean shrimp;
撒母耳阿。
It was he.
我保证。
Target Line;
没有应问。
Chemical potential.
上校：嗯。
He ate no soup.
他成功了。
What is to be done?
多么讽刺！。
Front wheel drive.
撒母耳阿。
All streams flow into the Huanghe River.
车船齐发。
A:It's on the seccond floor.
在二楼.。
All the boats and carts started off at the same time.
车船齐发。
The Univ.
没有应问。
The printing plant.
没有应问。
It looks like rain.
我保证。
Straight tool holder .
撒母耳阿。
I'm afraid of dying a violent death.
我怕暴死。
How do they channel it?
如何引导？。
You love me, then?
抱紧我。
The fragrance of flowers assails one's nose.